### Retrieve from MountainHub

This code allow for direct access to the MountainHub API. Data are returned in a Pandas dataframe.

Some code is adopted from Don Setiawan's work stored in the [cso legacy api](https://github.com/communitysnowobs/cso-legacy-api).

A. Arendt 20191231


In [3]:
import sys
import time
import datetime
from flask_restful import fields, marshal, reqparse, Resource, inputs

import pandas as pd
import geopandas as gpd
import requests

BASE_URL = 'https://api.mountainhub.com/timeline'
HEADER = { 'Accept-version': '1' }
ONE_MONTH = 2592000000

Load these [utility functions from the cso-legacy-api](https://github.com/communitysnowobs/cso-legacy-api/blob/master/src/common/utils.py).

Todo: move these to a script after testing.

In [27]:
def empty_cso_dataframe():
    return pd.DataFrame(columns = [
        'author_name',
        'id',
        'timestamp',
        'lat',
        'long',
        'snow_depth',
        'source',
        'elevation'
    ])

def date_to_timestamp(date):
    """Converts datetime object to unix timestamp.
    Keyword arguments:
    date -- Datetime object to convert
    """
    if date is None:
        return date
    return int(time.mktime(date.timetuple())) * 1000

def timestamp_to_date(timestamp):
    """Converts unix timestamp to datettime object.
    Keyword arguments:
    timestamp -- Timestamp to convert
    """
    if timestamp is None:
        return timestamp
    return datetime.fromtimestamp(timestamp / 1000)

In [35]:
now_ts = date_to_timestamp(datetime.datetime.now())
start_ts = date_to_timestamp(datetime.datetime(2019,12,30))

Load parsing function from the [MountainHub cso-legacy-api script](https://github.com/communitysnowobs/cso-legacy-api/blob/master/src/resources/obs/MountainHub.py).

#### NOTE: the MountainHub API has different named fields than what we were previously using in the legacy script above.

The code below is an attempt to match what is now in the MountainHub API:

In [36]:
def parse_data(record):

    cso_format = {
        'author_name' : fields.String(attribute='actor.full_name') or fields.String(attribute='actor.fullName'),
        'id' : fields.String(attribute=lambda x: x['_id']),
        'timestamp' : fields.Integer(attribute=lambda x: x['observation']['reported_at']),
        'lat' : fields.Float(attribute=lambda x: x['location']['coordinates'][1]),
        'long' : fields.Float(attribute=lambda x: x['location']['coordinates'][0]),
        'snow_depth' : fields.Float(attribute=lambda x: x['observation']['details'][0]['snowpack_depth']),
        'source' : fields.String(default = 'MountainHub')}

    return marshal(record, cso_format)

In [37]:
def fetch_raw_data(min_timestamp, max_timestamp, is_raw_json=False):

    args = {
      'publisher': 'all',
      'limit': 10000,
      'since' : min_timestamp,
      'before' : max_timestamp,
    }

    response = requests.get(BASE_URL, params=args, headers=HEADER)
    data = response.json()

    if 'results' not in data or len(data['results']) == 0:
        return empty_cso_dataframe()
    else:
        results = data['results']
        if is_raw_json:
            return results
        all_results = [ parse_data(result) for result in results ]
        valid_results = [result for result in all_results if result is not None]
        valid_results = with_elevation(valid_results)
        if len(valid_results) == 0:
            return empty_cso_dataframe()

        df = pd.DataFrame.from_records(valid_results)
        return df

### This throws an error

Seems to be a key error with some of the fields

In [38]:
df = fetch_raw_data(start_ts, now_ts )

KeyError: 'observation'

If we return the raw json, we can dig deeper:

In [39]:
rawdata = fetch_raw_data(start_ts, now_ts, is_raw_json = True )

This has two sets of MountainHub entries. Looking at the first one:

In [50]:
rawdata[0]['observation']

KeyError: 'observation'

And now the second one:

In [53]:
rawdata[1]['observation']

{'details': [{'snowpack_depth': '35'}],
 'media': [],
 'is_removed': False,
 'is_published': False,
 '_id': '5e0a4cc13c97304f4e8bdd3f',
 'type': 'snow_conditions',
 'location': [-120.39883648693286, 48.60518452553656],
 'user': '5889d368c22f143e609da787',
 'place': {'location': {'type': 'Point',
   'coordinates': [-120.39883648693286, 48.60518452553656]},
  '_id': '5e0a4cc13c97304f4e8bdd3e',
  'name': 'National Forest Development Road 060, Mazama, WA, USA'},
 'source': 1,
 'fingerprint': 'C819375A-35D5-4068-8912-4D99B6CD629A',
 'sharing': {'organizations': [],
  'is_pro': False,
  'is_public': True,
  'is_private': False,
  '_id': '5e0a4cc13c97304f4e8bdd40'},
 'reported_at': 1577733250329,
 'created_at': 1577733313582,
 'updated_at': 1577733313582,
 '__v': 0}

In [54]:
rawdata[1]['observation']['reported_at']

1577733250329

In [55]:
rawdata[1]['observation']['details'][0]['snowpack_depth']

'35'

So code needs to be modified to handle cases where there is no 'observation' field.